Загрузка постов с facebook
------------

Константы, используемые в проекте

In [29]:
def constant(func):
    def funcset(self, value):
        raise TypeError
    def funcget(self):
        return func()
    return property(funcget, funcset)

class _Const(object):
    @constant
    def FACEBOOK_TOKEN():
        return "1769775703259571|736fc7f9c5dc31707d40709a1d37813b"
 
    @constant
    def FACEBOOK_POSTS_COUNT():
        return 2000
    
    @constant
    def ES_INDEX_PREFIX():
        return "fb_group_posts_"
    
    @constant
    def ES_POSTS_DOC_TYPE():
        return "post"

CONST = _Const()

In [30]:
# coding=utf-8
from facebook import GraphAPI
from elasticsearch import Elasticsearch

import time as t
import requests

Создаем подключение к базе данных

In [31]:
database = Elasticsearch()

Список id-ников груп, для загрузки постов

In [32]:
groups_ids = [
    "7155422274",       # cnet
    "338028696036",     # yahoo news
    "32204506174",      # city news Toronto
    "122177661170978",  # news one
    "235852889908002",  # vice news
    "7245371700",       # MTV
    "179035672287016",  # American news
]

Создаем объект графа для работы с API от facebook

In [33]:
graph = GraphAPI(access_token=CONST.FACEBOOK_TOKEN)

Функция возвращающая текущее время

In [17]:
def time():
    return int(round(t.time() * 1000))

Вытаскиваем с пака постов текст сообщений

In [18]:
def extract_posts_data(posts):
    res = []
    
    for p in posts['data']:
        res.append(p)
        
    return res

Подготавливаем ссылку для GET запроса (извлекая его из пака постов) для загрузки следующей страницы

In [19]:
def prepare_get_posts_request(posts):
    return posts['paging']['next'].replace("limit=25", "limit=100")

Последовательно загружаем посты от указанного пака

In [20]:
# noinspection PyShadowingNames
def load_posts_pages(posts, max_count):
    res = []
    count = 0

    while True:
        if count > max_count:
            break

        try:
            res.extend(extract_posts_data(posts))

            request = prepare_get_posts_request(posts)

            s_time = time() 
            posts = requests.get(request).json()
            f_time = time()

            posts_count = len(posts['data'])
            count += posts_count

            print "load page time:", (f_time - s_time), ", page size:", posts_count, ", all count: ", count
        except KeyError:
            break

    print "extracted posts count: ", count
    print "--------------------------------"

    return res

Сохранение постов в базу данных

In [34]:
def save_posts(index, posts):
    for post in posts:
        database.index(index=index, doc_type=CONST.ES_POSTS_DOC_TYPE, id=post['id'], body=post)

Вытаскиваем имя группы с данных о группе

In [22]:
def extract_group_name(group):
    return group['name'].decode('utf-8').lower().replace(" ", "_")

Сохранение постов группы

In [23]:
# noinspection PyShadowingNames
def save_group_posts(graph, group_id):
    s_time = time()
    group = graph.get_object(group_id)
    f_time = time()

    print "load group data time: ", str(f_time - s_time)

    group_name = extract_group_name(group)

    print "downloading posts from: ", group_name

    posts = graph.get_connections(group['id'], 'feed')
    posts_data = load_posts_pages(posts=posts, max_count=CONST.FACEBOOK_POSTS_COUNT)
    index = CONST.ES_INDEX_PREFIX + group_name + "_" + group_id
    save_posts(index, posts_data)
  

Запускаем процесс загрузки постов по заготовленным id групп

In [24]:
for group_id in groups_ids:
    save_group_posts(graph=graph, group_id=group_id)

load group data time:  594
downloading posts from:  cnet
load page time: 2358 , page size: 100 , all count:  100
load page time: 2730 , page size: 100 , all count:  200
load page time: 1521 , page size: 100 , all count:  300
load page time: 1831 , page size: 100 , all count:  400
load page time: 1990 , page size: 100 , all count:  500
load page time: 1296 , page size: 100 , all count:  600
load page time: 1898 , page size: 100 , all count:  700
load page time: 1646 , page size: 100 , all count:  800
load page time: 2377 , page size: 100 , all count:  900
load page time: 1906 , page size: 100 , all count:  1000
load page time: 1597 , page size: 100 , all count:  1100
load page time: 2345 , page size: 100 , all count:  1200
load page time: 3178 , page size: 100 , all count:  1300
load page time: 2236 , page size: 100 , all count:  1400
load page time: 2541 , page size: 100 , all count:  1500
load page time: 2662 , page size: 100 , all count:  1600
load page time: 2995 , page size: 100 , 